<a href="https://colab.research.google.com/github/Shujaat123/ACP_Generator/blob/main/ACP_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import Dataset

In [ ]:
## import libraries
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

import os
import tensorflow as tf
import numpy as np
import pandas as pd

TensorFlow 2.x selected.


In [ ]:
## read acp240 dataset from https://github.com/haichengyi/ACP-DL/blob/master/acp240.txt
## read acp740 dataset from https://github.com/haichengyi/ACP-DL/blob/master/acp740.txt
acp240 = pd.read_csv('https://raw.githubusercontent.com/haichengyi/ACP-DL/master/acp240.txt', header=None)
acp740 = pd.read_csv('https://raw.githubusercontent.com/haichengyi/ACP-DL/master/acp740.txt', header=None)


In [ ]:
list(acp240[0][0])

## preprocess acp240
acp240_id = []
acp240_seq = []
for text in acp240[0]:
  if ('>' in list(text)):
    acp240_id.append(text)
  else:
    acp240_seq.append(text)

## preprocess acp740
acp740_id = []
acp740_seq = []
for text in acp740[0]:
  if ('>' in list(text)):
    acp740_id.append(text)
  else:
    acp740_seq.append(text)

## check results
print(acp240_seq)
print(acp740_seq)
print(len(acp240_seq))
print(len(acp740_seq))
print(max(len(i) for i in acp240_seq))
print(min(len(i) for i in acp240_seq))
print(max(len(i) for i in acp740_seq))
print(min(len(i) for i in acp740_seq))

['AIGSILGALAKGLPTLISWIKNR', 'AWKKWAKAWKWAKAKWWAKAA', 'FAKALAKLAKKLL', 'FAKFLAKFLKKAL', 'FAKIIAKIAKIAKKIL', 'FAKKFAKKFKKFAKKFAKFAFAF', 'FAKLLAKAFKKAL', 'FAKLLAKALKLKL', 'FAKLLFKALKKAL', 'FAKLWAKLAKKL', 'FALAKALKKAL', 'FALALKALKKALKKLKKALKKAL', 'FALLKALKKAL', 'FKRLAKIKVLRLAKIKR', 'FLGALFHALSKLL', 'FLGALFKALSHLL', 'GFFALIPKIISSPLFKTLLSAVGSALSSSGGQE', 'GIGKFLHAAKKFAKAFVAEIMNS', 'GIGKFLHSAKKWGKAFVGQIMNC', 'GKWKKILGKLIR', 'GKWKSLLKHILK', 'GKWLSLLKHILK', 'GKWMKLLKHILK', 'GKWMSFLKHILK', 'GKWMSLLKKILK', 'GKWMSLWKHILK', 'GKWMTLLKHILK', 'GLFAVIKKVAAVIKKL', 'GLFAVIKKVASVIKGL', 'GLFGKLIKKFARKAISYAVKKARGKH', 'GMWSKILKHLIR', 'HHPHGHHPHGHHPHGHHPHG', 'ILRWPWWPWRRK', 'KIAKVALAKLGIGAVLKVLTTGL', 'KKKFPWWWPFKKKCKKKFPWWWPFKKKC', 'KWKKFLKIGIGKFLHLAKKF', 'KWKLAKKALALL', 'KWKLFAKIGIGKFLHLAKKF', 'KWKLFKKIGIGAFLHLAKKF', 'KWKLFKKIGIGAVLKVLKKG', 'KYKKALKKLAKLL', 'MPRWRLFRRIDRVGKQIKQGILRAGPAIALVGDARAVG', 'MRKWFHNVLSSGQLLADKWPAWDYNRK', 'MWKWFHNVLSSGQLLADKWWAWWYNWW', 'NKWKKILGKIIKVVK', 'PDEDAINDALNKVCSTGRRQRSICKQLLKK

In [ ]:
## seq_to_text
acp240_txt = '\n'.join(acp240_seq)
acp740_txt = '\n'.join(acp740_seq)
acp_txt = acp240_txt + '\n' + acp740_txt
acp_txt

'AIGSILGALAKGLPTLISWIKNR\nAWKKWAKAWKWAKAKWWAKAA\nFAKALAKLAKKLL\nFAKFLAKFLKKAL\nFAKIIAKIAKIAKKIL\nFAKKFAKKFKKFAKKFAKFAFAF\nFAKLLAKAFKKAL\nFAKLLAKALKLKL\nFAKLLFKALKKAL\nFAKLWAKLAKKL\nFALAKALKKAL\nFALALKALKKALKKLKKALKKAL\nFALLKALKKAL\nFKRLAKIKVLRLAKIKR\nFLGALFHALSKLL\nFLGALFKALSHLL\nGFFALIPKIISSPLFKTLLSAVGSALSSSGGQE\nGIGKFLHAAKKFAKAFVAEIMNS\nGIGKFLHSAKKWGKAFVGQIMNC\nGKWKKILGKLIR\nGKWKSLLKHILK\nGKWLSLLKHILK\nGKWMKLLKHILK\nGKWMSFLKHILK\nGKWMSLLKKILK\nGKWMSLWKHILK\nGKWMTLLKHILK\nGLFAVIKKVAAVIKKL\nGLFAVIKKVASVIKGL\nGLFGKLIKKFARKAISYAVKKARGKH\nGMWSKILKHLIR\nHHPHGHHPHGHHPHGHHPHG\nILRWPWWPWRRK\nKIAKVALAKLGIGAVLKVLTTGL\nKKKFPWWWPFKKKCKKKFPWWWPFKKKC\nKWKKFLKIGIGKFLHLAKKF\nKWKLAKKALALL\nKWKLFAKIGIGKFLHLAKKF\nKWKLFKKIGIGAFLHLAKKF\nKWKLFKKIGIGAVLKVLKKG\nKYKKALKKLAKLL\nMPRWRLFRRIDRVGKQIKQGILRAGPAIALVGDARAVG\nMRKWFHNVLSSGQLLADKWPAWDYNRK\nMWKWFHNVLSSGQLLADKWWAWWYNWW\nNKWKKILGKIIKVVK\nPDEDAINDALNKVCSTGRRQRSICKQLLKK\nPEWFKCRRWQWRMKKLGA\nDSHEERHHGRHGHHKYGRKFHEKHHSHRGYRSNYLYDN\nALSILKGLEKLAKMGIALTNCKATKKC\n

In [ ]:
vocab = sorted(set(acp_txt))
print ('{} unique characters'.format(len(vocab)))
print(vocab)

21 unique characters
['\n', 'A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y']


# Preprocess Sequences

Create integer sequence represents AA sequence

In [ ]:
## Unique characters in the sequeces
aa_dict = sorted(set(acp_txt)) ### Sort index of iterables (enumerate(aa))

## Amino Acid to Index
aa2idx = {u:i for i, u in enumerate(aa_dict)}

## Index to Amino Acid
idx2aa = np.array(aa_dict)

## Create AA sequence as integer indices
aa_as_int = np.array([aa2idx[u] for u in acp_txt]) # don't forget bracket '[]' to make it accessible.

for aa,_ in zip(aa2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(aa), aa2idx[aa]))
print('  ...\n}')
##########################################################################################################################
## Check whether created object working fine
# print(aa_dict)
# print(len(aa_dict))
# print(aa2idx) ## AA dictionary
# print(aa2idx['A']) ### Index of AA 'A'
# print(idx2aa) ## all AAs
# print(np.array2string(idx2aa[2])) ### 2nd index is 'A'

# print('{')
# for aa,_ in zip(aa2idx, range(20)):
#     print('  {:4s}: {:3d},'.format(repr(aa), aa2idx[aa]))
# print('  ...\n}')

# print ('{} ---- characters mapped to int ---- > {}'.format(repr(afp_seq[:13]), aa_as_int[:13]))

# test = aa_as_int ## save object to test
# test = test.tolist() ## unpack np.ndarray to list
# print(test) ### integer sequence that represent AAs for all sequences, while seperated by '\n' and '\r'.
# print(len(test)) ### Check whether the returned value is the same as len(afp_seq)
# print(idx2aa[test]) ### integer sequence that represent AAs for all sequences, while seperated by '\n' and '\r'.
# print(afp_seq[:3]) ### Check the original afp_seq
##########################################################################################################################

  '\n':   0,
  'A' :   1,
  'C' :   2,
  'D' :   3,
  'E' :   4,
  'F' :   5,
  'G' :   6,
  'H' :   7,
  'I' :   8,
  'K' :   9,
  'L' :  10,
  'M' :  11,
  'N' :  12,
  'P' :  13,
  'Q' :  14,
  'R' :  15,
  'S' :  16,
  'T' :  17,
  'V' :  18,
  'W' :  19,
  ...
}


Create training dataset with desired sequence length

In [ ]:
## Desired seq length for input data
seq_length = 64
examples_per_epoch = len(acp_txt)//(seq_length+1) ## '//' division & floor (내림)

## Create tensorflow dataset
aa_dataset = tf.data.Dataset.from_tensor_slices(aa_as_int)

## Create batch dataset using batch method in tf.Dataset class
seq_batch = aa_dataset.batch(seq_length+1, drop_remainder=True) ## drop the smaller batch

for item in seq_batch.take(5): ## To see all dataset, take(-1)
  print(repr(''.join(idx2aa[item.numpy()])))

##########################################################################################################################
## Check numbers
# print(len(afp_seq))
# print(seq_length+1)
# print(examples_per_epoch)
# print(len(afp_seq)/(seq_length+1))

## Check dataset
# for i in aa_dataset.take(5):  ## faster
#   print(idx2aa[i.numpy()])

# print(list(aa_dataset.as_numpy_iterator())[:177]) ## very slow
# print(np.array2string(idx2aa[list(aa_dataset.as_numpy_iterator())][:177])) ## very slow too
##########################################################################################################################

'AIGSILGALAKGLPTLISWIKNR\nAWKKWAKAWKWAKAKWWAKAA\nFAKALAKLAKKLL\nFAKFL'
'AKFLKKAL\nFAKIIAKIAKIAKKIL\nFAKKFAKKFKKFAKKFAKFAFAF\nFAKLLAKAFKKAL\nF'
'AKLLAKALKLKL\nFAKLLFKALKKAL\nFAKLWAKLAKKL\nFALAKALKKAL\nFALALKALKKALK'
'KLKKALKKAL\nFALLKALKKAL\nFKRLAKIKVLRLAKIKR\nFLGALFHALSKLL\nFLGALFKALS'
'HLL\nGFFALIPKIISSPLFKTLLSAVGSALSSSGGQE\nGIGKFLHAAKKFAKAFVAEIMNS\nGIG'


# Create training dataset

prepare input and target sequence of model

In [ ]:
def split_input_target(chunk):
    input_seq = chunk[:-1]
    target_seq = chunk[1:]
    return input_seq, target_seq

dataset = seq_batch.map(split_input_target) # apply function for each item in seq_batch
dataset
#########################################################################################################################################
# ## Check Input and target sequence of RNN model
# for input, target in dataset.take(1):
#   print('input: ', repr(''.join(idx2aa[input.numpy()])))
#   print('target:', repr(''.join(idx2aa[target.numpy()])))

# ## Check chain of sequence
# for i, (input_idx, target_idx) in enumerate(zip(input[:10], target[:10])):
#   print("step:{:2d}, {} ({:s}) -->  {} ({:s})".format(i, input_idx, repr(idx2aa[input_idx]), target_idx, repr(idx2aa[target_idx])))
#########################################################################################################################################

<MapDataset shapes: ((64,), (64,)), types: (tf.int64, tf.int64)>

create batch dataset

In [ ]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True) # Do shuffling and select 64 items from 1024 items for each
dataset

<BatchDataset shapes: ((64, 64), (64, 64)), types: (tf.int64, tf.int64)>

# Build Model (ACP-Generator)

In [ ]:
## Length of AA Dictionary
aa_dict_size = len(aa_dict)

## Embedding dimension
embedding_dim = 256 ## Embedding each AA into 256 dimensional vector
# embedding_dim = 128 ## reserve for testing

# Number of RNN units
rnn_units = 1024
# rnn_units = 512 ## reserve for testing

In [ ]:
def build_model(aa_dict_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(aa_dict_size, embedding_dim, batch_input_shape = [batch_size, None]),
    tf.keras.layers.GRU(rnn_units, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform', reset_after=True),
    tf.keras.layers.GRU(rnn_units, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform', reset_after=True),
    tf.keras.layers.Dense(aa_dict_size)
  ])
  return model

In [ ]:
model = build_model(
    aa_dict_size = len(aa_dict),
    embedding_dim = embedding_dim,
    rnn_units = rnn_units,
    batch_size = BATCH_SIZE
)

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           5376      
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3938304   
_________________________________________________________________
gru_1 (GRU)                  (64, None, 1024)          6297600   
_________________________________________________________________
dense (Dense)                (64, None, 21)            21525     
Total params: 10,262,805
Trainable params: 10,262,805
Non-trainable params: 0
_________________________________________________________________


# Check the Model

In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 64, 21) # (batch_size, sequence_length, vocab_size)


# Create Random Sequence for Comparision (Sampled from Uniform Distribution)

In [ ]:
def generate_random_text(min_thres=10, max_thres=250):
  # Evaluation step (generating text using the learned model)
  start_string=u"\n"
  # Number of AAs to generate
  num_generate = 5000

  # Converting our start string to numbers (vectorizing)
  input_eval = [aa2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  for i in range(num_generate):
      predicted_id = np.random.randint(0,21)
      # We pass the predicted word as the next input to the model
      # along with the previous hidden state
      text_generated.append(idx2aa[predicted_id])
      if(text_generated[-1] == start_string):
        text_generated.append

  text_unfiltered = ''.join(text_generated)
  text_unfiltered = text_unfiltered.splitlines()

  # Empty string to store our results
  text_filtered = []
  for j in range(len(text_unfiltered)):
    if(len(text_unfiltered[j]) < min_thres):
      next
    elif(len(text_unfiltered[j]) > max_thres):
      next
    else:
      text_filtered.append('>' + start_string + ''.join(text_unfiltered[j])+'\n')

  return (''.join(text_filtered))

In [ ]:
print(generate_random_text())

>
VWTMTRLSAESTKEPRTQSHGKVNTE
>
ITKDFDDDHLSVYWI
>
NCFRCDPVSSDGKAETGIDVWNYEHEGTHNVQKQQHNDCDYKGAGADFGNCKAPRGHHWDMM
>
LMCTGGKFTFTLSWKENTCLWTIAVNMCGTPDMYQMPNISSEGQDYYRLTMDWDQYTHVMPFDE
>
AQCPGFWNRRTSKQP
>
EAEWCMVYYPNGLECWEPTWQNRQVVHRWHRASEQHWATFPFVFYVKKFVGVTLV
>
CAGKWWVDGHTKEMTWIEMHAIPSMPTGFYYQPQWFIWNTIDVFAKHDDNGFEIWVMLHGHQAYFAP
>
IENYRMGCGSTDDTKAYWTY
>
TYICLDTLWPQASFGEGLWTPAEDHVQKQRTNRIDGLWYYDRRTFPNYYDGFMQMDHMQNKMQVNAFPRQNTAWRAYIKDQQFQWCTRRLQPAE
>
YERRMTFHCMNLR
>
EDTILFEESANDFMSKYRIWA
>
MMQSECLWWKNIPGLQYIFVFS
>
TGEEQLPENKVKPEVQWNNYSGDVIFISSHRYCFMGDY
>
DYYHNMSKRNEWLGGPPNYNAYN
>
IGSADSLFEKL
>
DDKMDMVMQMWNFNLEKDMDNHGYCFMTCDNVWA
>
TTVMYDSFSNINW
>
SGDLMLECTGSFEMMMWTEYGPGRGCQEF
>
RATYGKVNTR
>
QIMCTGWARHHMTPHRFKA
>
KGSYRSNRTNPWWV
>
DNNAPLWCTPIFLWIDCWHARGWISMNFTPWSAEPGSDAKKQPWGVMWWCNHLAVYSDQRKFV
>
GHVVYEIVHDQQRWHVWEAAQSTETFRDKYMEHIRI
>
SSMMCWKSCPTAQHCVSSQTIMCPIRFNSTPFPWGEAVKACPHHISYNNSGL
>
MYCSRHMQRVVDHLVFYMGMMNTITQQPG
>
GKGSPKFGIMDWN
>
HVMYPTSTHDSNIEAHVYTELKSDT
>
SEDYRAAKHHGEMGQYCNIMLR
>
HDTQMTVLF

# Training

Define loss function

In [ ]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 64, 21)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       3.0451279


Compile model

In [ ]:
model.compile(optimizer='adam', loss=loss)

Configure checkpoints

In [ ]:
from keras.callbacks import EarlyStopping

# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'

# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")
es = EarlyStopping(monitor='loss', mode='min', verbose=0, patience=25)

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

Using TensorFlow backend.


Execute Training

In [ ]:
EPOCHS=100
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback, es], verbose=2)

Train for 6 steps
Epoch 1/100
6/6 - 2s - loss: 3.1925
Epoch 2/100
6/6 - 1s - loss: 2.9914
Epoch 3/100
6/6 - 1s - loss: 2.9273
Epoch 4/100
6/6 - 1s - loss: 2.8668
Epoch 5/100
6/6 - 1s - loss: 2.8595
Epoch 6/100
6/6 - 1s - loss: 2.8482
Epoch 7/100
6/6 - 1s - loss: 2.8283
Epoch 8/100
6/6 - 1s - loss: 2.8148
Epoch 9/100
6/6 - 1s - loss: 2.8113
Epoch 10/100
6/6 - 1s - loss: 2.7996
Epoch 11/100
6/6 - 1s - loss: 2.7884
Epoch 12/100
6/6 - 1s - loss: 2.7782
Epoch 13/100
6/6 - 1s - loss: 2.7643
Epoch 14/100
6/6 - 1s - loss: 2.7463
Epoch 15/100
6/6 - 3s - loss: 2.7377
Epoch 16/100
6/6 - 6s - loss: 2.7212
Epoch 17/100
6/6 - 2s - loss: 2.7055
Epoch 18/100
6/6 - 4s - loss: 2.6906
Epoch 19/100
6/6 - 2s - loss: 2.6769
Epoch 20/100
6/6 - 3s - loss: 2.6717
Epoch 21/100
6/6 - 3s - loss: 2.6589
Epoch 22/100
6/6 - 3s - loss: 2.6470
Epoch 23/100
6/6 - 3s - loss: 2.6279
Epoch 24/100
6/6 - 3s - loss: 2.6242
Epoch 25/100
6/6 - 2s - loss: 2.5962
Epoch 26/100
6/6 - 2s - loss: 2.5754
Epoch 27/100
6/6 - 6s - loss:

*Generate* ACP Sequences

# Generate text

Restore the lastest checkpoint

In [ ]:
# restore checkpoint
tf.train.latest_checkpoint(checkpoint_dir)
# load model
model = build_model(aa_dict_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

# summary
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            5376      
_________________________________________________________________
gru_2 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
gru_3 (GRU)                  (1, None, 1024)           6297600   
_________________________________________________________________
dense_1 (Dense)              (1, None, 21)             21525     
Total params: 10,262,805
Trainable params: 10,262,805
Non-trainable params: 0
_________________________________________________________________


Generate Text

In [ ]:
def generate_text(model, start_string, min_thres=10, max_thres=250):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 5000

  # Converting our start string to numbers (vectorizing)
  input_eval = [aa2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the word returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted word as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2aa[predicted_id])
      if(text_generated[-1] == start_string):
        text_generated.append

  text_unfiltered = ''.join(text_generated)
  text_unfiltered = text_unfiltered.splitlines()

  # Empty string to store our results
  text_filtered = []
  for j in range(len(text_unfiltered)):
    if(len(text_unfiltered[j]) < min_thres):
      next
    elif(len(text_unfiltered[j]) > max_thres):
      next
    else:
      text_filtered.append('>' + start_string + ''.join(text_unfiltered[j])+'\n')

  return (''.join(text_filtered))

In [ ]:
print(generate_text(model, start_string=u"\n"))

>
MFITCDLLSAFGVGHAACAAHCIGHGYRGGYCNSKAVCTCRR
>
DDTPSSRCGSGGWGPCLPIVDLLCIGKRRRRRRRR
>
FLPKIFCAITKKC
>
ATCDLLSGTGINHSACAAHCLLRGNRGGYCNGAAICVWN
>
FLPLIAGLAANFLPKLFCKITKKC
>
AISYGNGVYCNKEKCWVNKAENKQAITGRPSGLSLEERLQPRLSEA
>
ALTMNHFTALPKLLPLALGALKNLIK
>
IIEKLVNTALGLLSGL
>
IIGLVSKGTCVLVKTVCKKVLKQG
>
IKITTMLLGANALKGLYDNFQELVSSIEAALTKALGLCGTVSFGHAIG
>
LILATVPFEELQLALLERQTVFL
>
GHGRLVEIQGLTRKIRLHGLIR
>
GKWMSLWKHILK
>
GKWMSLLKHILK
>
GFGMALKLLKKVLPKVMKHF
>
FASLLAKALKLKL
>
FAKLLFKALKKAL
>
FAKLWAKLAKKL
>
FALAKALKKAL
>
FALLKALKKAL
>
FKRLAKIKVLRLAKIKR
>
FLGALFHALSKLL
>
FLGALFKALAKLAKKL
>
KLLKLWLKYWQNTR
>
TEVLLAGFLFLVTVCKNHEFCPGDLMSEC
>
FLIGIVHGVICSITKKC
>
CGESCVFIPCISTLLGCSCKNKVCYRNGVIP
>
FLPIVTGLLSGLL
>
FLPLVTGLLSGLL
>
CGESCVWIPCVTSIFNCKCKENKVCYHD
>
FIGSIVGALASLLGK
>
GLRGKILRSAIKQAGKAGLEELFRKAADKISTWKNVGKAAGKAVLNTVTMMVPQVWWSIVRRACRVLDTVKCKVTGAC
>
FLSLIPHIVSGVASIAKHF
>
AEVAPAPAAAPAKAPKKKAAAKPKKAGKKVLPKAAKAGKAFAKIINIARMRRAWDRNVYKKAQPCVPYCFNRRGRSREAAQRGYIYKKYPKGAKVPNKVKMLVNIRGKQTMRTCYLMSWTASSRTAKYYYYI
>